# 小町算の解を探索したい
[解きたい問題](https://twitter.com/furusatojuku/status/1458234116260827136?s=20)

---
## 問題
N個の与えられた数をもとに`+, -, *, /`の四則演算だけを用いて数Kを作ることができるか判定する。
今回は問題の条件に則ってN=4とする。

### 制約
```
N == 4
1 <= A_i <= 9 (1 <= i <= N)
```

### 入力
```
N K
A_1 A_2 ... A_N
```

### 出力
条件を満たす場合は`Yes`、満たさない場合は`No`を出力せよ。
また、条件を満たす場合には式の1例をスペース区切りのRPN形式で出力せよ。

### 例
**Input**
```
4 10
3 4 5 6
```

**Output**
```
Yes
3 4 * 5 * 6 /
```

## 方針
普通の式を扱おうとすると、カッコを処理する必要がありめんどくさい。
逆ポーランド記法を用いることで、簡単に実装できるはず。

1. N個の数の順列を生成 → N! : (4! = 24)
1. N個の数を処理するため、N-1個の演算子を生成 → (N-1)^4 : ((4-1)^4 = 81)
1. RPNで処理
1. 結果がKである場合停止して出力
1. 全探索が終了したらNoを出力

合計のループ回数は、24*81 = 1944 余裕はある。

In [1]:
from itertools import permutations, product
from operator import add, sub, mul, truediv

In [2]:
# RPNパーサ
# [a, b, c, d], [op1, op2, op3]
# -> [a, b, op3(d, c)], [op1, op2]
# -> [a, op2(op3(d, c), b)], [op1]
# -> [op1(op2(op3(d, c), b), a)]
# -> op1(op2(op3(d, c), b), a)

func = {
    "+": add,
    "-": sub,
    "*": mul,
    "/": truediv,
}

def parse_rpn(nums, ops) -> int:
    """
    argments
    ---------------------
    nums : list[int]
    ops  : list[callable]

    returns
    ---------------------
    res  : int
    """

    res = nums.pop()
    while nums:
        n = nums.pop()
        op = ops.pop()
        res = func[op](res, n)
        
    return res


In [3]:
n = [6, 5, 4, 3]
op = ["/", "*", "*"]
parse_rpn(n, op)

10.0

In [5]:

N, K = map(int, input().split())
A = list(map(int, input().split()))

permN = map(list, permutations(A))  # 重複のある順列も列挙されてしまうが、N=4で固定なので無視する
operators = ["+", "-", "*", "/"]
op_combi = map(list, product(*[operators]*3))

for nums, ops in product(permN, op_combi):
    res = parse_rpn(nums[:], ops[:])
    if res == K:
        print("Yes")
        # 文字列を出力
        out = str(nums.pop())
        while nums:
            out += " " + str(nums.pop())
            out += " " + ops.pop()

        print(out)
        break
else:
    print("No")

Yes
4 3 + 2 + 1 +


## 失敗
**Input**
```
4 10
3 4 7 8
```

```
-> No
```

答えである
```
3 7 4 / - 8
```

を認識できなかったのが問題

## 再度挑戦
演算子と数を混ぜた7個の順列を取る。
合計のループ回数は、4^3 * 7! = 322560、結構きわどい

In [6]:
# 改良版
# [3, 7, 4, "/", "-", 8, "*"]
# -> ["*", 8, "-", "/", 4, 7, 3]
# 数はスタックに保存、演算子が来たらスタックから取り出して処理

func = {
    "+": add,
    "-": sub,
    "*": mul,
    "/": truediv,
}

def parse_rpn2(f) -> int:
    """
    argments
    ---------------------
    f    : list[int, str]

    returns
    ---------------------
    res  : int
    """
    f.reverse()
    stack = []
    while f:
        try:
            item = f.pop()
            if isinstance(item, int):
                stack.append(item)
            else:
                a, b = stack.pop(), stack.pop()
                temp = func[item](b, a)
                stack.append(temp)
        except:  # うまくいかない場合は無視
            return None
        
    return stack[0]

In [7]:
parse_rpn2([3, 7, 4, "/", "-", 8, "*"])

10.0

In [10]:
N, K = map(int, input().split())
A = list(map(int, input().split()))

# N, K = 4, 10
# A = [3, 4, 7, 8]

operators = ["+", "-", "*", "/"]
op_combi = map(list, product(*[operators]*3))

flag = False
for ops in op_combi:
    items = ops + A
    for f in permutations(items):
        res = parse_rpn2(list(f))
        if res == K:
            flag = True
            ok = f[:]

if flag:
    print("Yes")
    print(" ".join(map(str, ok)))
else:
    print("No")

Yes
4 1 3 / / 2 -


## 成功
↓最終的なコード

In [9]:
%%time

func = {
    "+": add,
    "-": sub,
    "*": mul,
    "/": truediv,
}

def parse_rpn2(f) -> int:
    f.reverse()
    stack = []
    while f:
        try:
            item = f.pop()
            if isinstance(item, int):
                stack.append(item)
            else:
                a, b = stack.pop(), stack.pop()
                temp = func[item](b, a)
                stack.append(temp)
        except:  # うまくいかない場合は無視
            return None
    return stack[0]

if __name__ == "__main__":  
    # N, K = map(int, input().split())
    # A = list(map(int, input().split()))
    N, K = 4, 10
    A = [3, 4, 7, 8]

    operators = ["+", "-", "*", "/"]
    op_combi = map(list, product(*[operators]*3))

    flag = False
    for ops in op_combi:
        items = ops + A
        for f in permutations(items):
            res = parse_rpn2(list(f))
            if res == K:
                flag = True
                ok = f[:]

    if flag:
        print("Yes")
        print(" ".join(map(str, ok)))
    else:
        print("No")

Yes
8 3 7 4 / - *
CPU times: user 2.78 s, sys: 0 ns, total: 2.78 s
Wall time: 2.78 s


↑AtCoderなら普通にTLEだけどまあいいでしょう。